In [ ]:
# ESTE ES UN RESUMEN DE LA CREACION DE LA BASE DE DATOS

In [ ]:
#!pip install python-dotenv

Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: C:\Users\nevaz\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [2]:
import os 
from sqlalchemy import create_engine

usuario=os.environ.get("USUARIO")
password=os.environ.get("PASSWORD")
puerto=os.environ.get("PUERTO")
servidor=os.environ.get("SERVIDOR")
db=os.environ.get("BASE_DE_DATOS")

engine=create_engine(f"mysql+pymysql://{usuario}:{password}@{servidor}:{puerto}/{db}")
con=engine.connect()

In [5]:
import os 
from sqlalchemy import create_engine

usuario=os.environ.get("USUARIO")
password=os.environ.get("PASSWORD")
puerto=os.environ.get("PUERTO")
servidor=os.environ.get("SERVIDOR")
db=os.environ.get("BASE_DE_DATOS")

engine=create_engine(f"mysql+pymysql://{usuario}:{password}@{servidor}:{puerto}/{db}")
con=engine.connect()


import pymysql
import pandas as pd

timeout = 10
connection = pymysql.connect(
  charset="utf8mb4",
  connect_timeout=timeout,
  cursorclass=pymysql.cursors.DictCursor,
  db=os.environ.get("BASE_DE_DATOS"),
  host=os.environ.get("SERVIDOR"),
  password=os.environ.get("PASSWORD"),
  read_timeout=timeout,
  port=11364,
  user=os.environ.get("USUARIO"),
  write_timeout=timeout,
)

# creamos el cursor.
crsr=connection.cursor()


def sql_query(query):

    # Ejecuta la query
    crsr.execute(query)

    # Almacena los datos de la query 
    ans = crsr.fetchall()

    # Obtenemos los nombres de las columnas de la tabla
    names = [description[0] for description in crsr.description]

    return pd.DataFrame(ans,columns=names)

In [ ]:
# una vez que creamos las conexiones necesarias vamos a crear las tablas que necesitamos para funcionar.
#tabla de AÑOS.
query="""
CREATE TABLE AÑOS
(      
    AÑOSID INT PRIMARY KEY AUTO_INCREMENT, 
    AÑO SMALLINT NOT NULL
);
"""
crsr.execute(query)

In [ ]:
# CREAMOS LA TABLA DE LAS PROVINCIAS

query="""
CREATE TABLE Provincia
(      
    COD_PROV INT PRIMARY KEY AUTO_INCREMENT, 
    PROVINCIA CHAR(20) NOT NULL
);"""

crsr.execute(query)

In [ ]:
#CREAMOS LA TABLA DE LOS CANDIDATOS ELECTORALES

query="""
CREATE TABLE CANDIDATO
(      
    CAND_ID INT AUTO_INCREMENT,
    CANDIDATO_COD INT NOT NULL,
    CANDIDATO_DESC TINYTEXT NOT NULL,
    CANDIDATO_SIGLAS CHAR(20) NOT NULL,
    AÑOID TINYINT NOT NULL,
    PRIMARY KEY(CAND_ID)
);"""

crsr.execute(query)


In [ ]:
#CREAMOS UNA TABLA QUE RELACIONE EL MUNICIPIO CON LA PROVINCIA, AUNQUE, DEBIDO A LOS DATOS QUE SE VAN A PASAR A LA BASE DE DATOS ES 
# ES POSIBLE QUE NO SE NECESITE REALIZAR. 

query="""
CREATE TABLE MUNICIPIO_PROVINCIA
(      
    COD_MUNICIPIO INT NOT NULL,
    MUNICIPIO TINYTEXT NOT NULL,
    COD_PROV INT NOT NULL,
    PRIMARY KEY(COD_MUNICIPIO)
);"""


crsr.execute(query)

In [ ]:
# CREAMOS LA TABLA CON LAS PROVINCIAS Y COMARCAS
query="""
CREATE TABLE COMARCAS
(      
    COD_MUNICIPIO INT NOT NULL,
    COD_COMARCA INT NOT NULL,
    COD_PROV INT NOT NULL,
    COMARCA TINYTEXT,
    PRIMARY KEY(COD_COMARCA)
);"""


crsr.execute(query)

In [ ]:
#TABLA QUE RELACIONA LA COMARCA CON EL MUNICIPIO.

query="""
CREATE TABLE COMARCA_MUNICIPIO
(      
    ID INT AUTO_INCREMENT PRIMARY KEY,
    COD_MUNICIPIO INT NOT NULL,
    COD_COMARCA INT NOT NULL,
    COMARCA TINYTEXT NOT NULL,
    MUNICIPIO TINYTEXT NOT NULL
);"""

crsr.execute(query)

In [ ]:
query="""
CREATE TABLE COMARCA_CENSO_VOTANTES
(      
    ID INT  AUTO_INCREMENT,
    COD_COMARCA INT,
    COMARCA TINYTEXT NOT NULL,
    CENSO INT NOT NULL,
    VOTANTES INT NOT NULL,
    PRIMARY KEY(ID)
    
);"""

crsr.execute(query)

In [ ]:
query="""CREATE TABLE VOTOS,
    (,
        ID INT AUTO_INCREMENT PRIMARY KEY,
        CANDIDATO_COD TINYINT NOT NULL,
        COD_COMARCA TINYINT NOT NULL,
        VOTOS INT NOT NULL,
        AÑOID TINYINT NOT NULL,
    );"""

crsr.execute(query)

In [ ]:
query="""
CREATE TABLE PARTIDOS_EUROPEOS
(      
    ID VARCHAR(15) NOT  NULL,
    LANGUAGE_ID VARCHAR(4) NOT NULL,
    ACRONYM VARCHAR(15) NOT NULL,
    LABEL TINYTEXT NOT NULL,
    TP TINYTEXT NOT NULL,
    AÑO_ID TINYINT NOT NULL,
    PRIMARY KEY(ID)
    
);"""

crsr.execute(query)

In [ ]:
query="""
CREATE TABLE PARTIDO_GRUPO
(      
    ID INT AUTO_INCREMENT PRIMARY KEY,
    CANDIDATO TINYTEXT NOT NULL,
    CANDIDATO_SIGLAS VARCHAR(15) NOT NULL,
    PARTIDOS TINYTEXT NOT NULL,
    PARTIDO_EUROPA TINYTEXT,
    AÑO_ID TINYINT NOT NULL
    
);"""

crsr.execute(query)

In [ ]:
#UNA VEZ CREADAS LAS TABLAS METEMOS LOS DATOS

df = pd.read_csv('años.csv')

# Inserta los datos en la tabla

#AÑOS
df.to_sql('AÑOS', con=engine, if_exists='append', index=False)

#PROVINCIA
df=pd.read_csv("provincia.csv").drop_duplicates().reset_index()
del df["index"]
df.to_sql('Provincia', con=engine, if_exists='append', index=False)

#CANDIDATOS
df=pd.read_csv("canditatos.csv")
df.to_sql('CANDIDATO', con=engine, if_exists='append', index=False)

#MUNICIPIO PROVINCIA
df=pd.read_csv("municipio_provincia.csv")

df.to_sql('MUNICIPIO_PROVINCIA', con=engine, if_exists='append', index=False)

#COMARCAS
df=pd.read_csv("comarcas.csv")
del df["PROVINCIA"]
query="""ALTER TABLE COMARCAS
DROP COLUMN COD_MUNICIPIO"""
crsr.execute(query)
df.to_sql('COMARCAS', con=engine, if_exists='append', index=False,)



#COMARCA_MUNICIPIO
df=pd.read_csv("comarca_municipio.csv")


df.to_sql('COMARCA_MUNICIPIO', con=engine, if_exists='append', index=False)

#COMARCA_CENSO_VOTANTES
df=pd.read_csv("comarcas_censo_votantes_sumado.csv")


df.to_sql('COMARCA_CENSO_VOTANTES', con=engine, if_exists='append', index=False,)

#PARTIDOS EUROPEOS
df=pd.read_csv("partidos_europeos.csv")


df.to_sql('PARTIDOS_EUROPEOS', con=engine, if_exists='append', index=False,)









In [ ]:
query="""ALTER TABLE CANDIDATO 
      DROP COLUMN AÑOID"""
crsr.execute(query)

query="""ALTER TABLE CANDIDATO 
      ADD COLUMN AÑOID TINYINT AFTER CANDIDATO_SIGLAS"""
crsr.execute(query)

query="""UPDATE CANDIDATO
      SET
      AÑOID = '3' """
crsr.execute(query)
connection.autocommit(True)


In [ ]:
#PARA LAS NUEVAS FILAS PODEMOS USAR EL MISMO METODO CON SQLALCHEMY Y NOS LAS AÑADE.
# NO OBSTANTE, HAY QUE TENER CUIDADO YA QUE, SIEMPRE QUE COINCIDAN LAS COLUMNAS SE AÑADIRAN LAS FILAS ADICIONALES.
query="DELETE FROM COMARCA_CENSO_VOTANTES WHERE ID >= 69;"
crsr.execute(query)
connection.autocommit(True)

# EN ESTE CASO FUE NECESARIO BORRAR LAS FILAS QUE YA ESTABAN HECHAS.